In [93]:
import sys, os
os.environ['SPARK_HOME'] = "D:\\ProgramFile\\spark-3.0.2-bin-hadoop2.7"
sys.path.append("D:\\ProgramFile\\spark-3.0.2-bin-hadoop2.7\\python")
sys.path.append("D:\\ProgramFile\\spark-3.0.2-bin-hadoop2.7\\python\\lib")
sys.path.append("D:\\ProgramFile\\spark-3.0.2-bin-hadoop2.7\\python\\pyspark")
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

from pyspark.sql.types import FloatType, IntegerType
import pyspark.sql.functions as fn
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer
import pandas as pd

conf = SparkConf().setMaster("local[8]").setAppName("process data")
spark = SparkSession.builder.config(conf = conf).getOrCreate()

In [106]:
df = spark.read.csv('D:\\pycharm_projects\\\RandomForest_Spark\\data\\bands.data', header=False)

该数据共有541个样本，40个特征

In [107]:
print((df.count(), len(df.columns)))
print(df.columns)

(541, 40)
['_c0', '_c1', '_c2', '_c3', '_c4', '_c5', '_c6', '_c7', '_c8', '_c9', '_c10', '_c11', '_c12', '_c13', '_c14', '_c15', '_c16', '_c17', '_c18', '_c19', '_c20', '_c21', '_c22', '_c23', '_c24', '_c25', '_c26', '_c27', '_c28', '_c29', '_c30', '_c31', '_c32', '_c33', '_c34', '_c35', '_c36', '_c37', '_c38', '_c39']


统计每列缺失值占比，可以发现缺失值占少数，可以考虑直接删除缺失值

In [108]:
df.agg(*[
    (1 - (fn.count(c) / fn.count('*'))).alias(c + '_missing')
    for c in df.columns
]).show()

+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+------------+------------+------------+------------+------------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|_c0_missing|_c1_missing|_c2_missing|_c3_missing|_c4_missing|_c5_missing|_c6_missing|_c7_missing|_c8_missing|_c9_missing|_c10_missing|_c11_missing|_c12_missing|_c13_missing|_c14_missing|_c15_missing|_c16_missing|        _c17_missing|        _c18_missing|        _c19_missing|        _c20_missing|        

In [109]:
df = df.dropDuplicates()

- 根据数据说明，前20个特征值是类别特征，后19个特征值是数值特征，最后一个是类别，删除第一个时间戳特征
- 直接去掉含缺失值的样本，剩余539个

In [110]:
for c in ["_c0", "_c1", "_c2", "_c3"]:
    df = df.drop(c)

In [115]:
df = df.dropna()

In [113]:
df.agg(*[
    (1 - (fn.count(c) / fn.count('*'))).alias(c + '_missing')
    for c in df.columns
]).show()

+-----------+-----------+-----------+-----------+-----------+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+
|_c4_missing|_c5_missing|_c6_missing|_c7_missing|_c8_missing|_c9_missing|_c10_missing|_c11_missing|_c12_missing|_c13_missing|_c14_missing|_c15_missing|_c16_missing|_c17_missing|_c18_missing|_c19_missing|_c20_missing|_c21_missing|_c22_missing|_c23_missing|_c24_missing|_c25_missing|_c26_missing|_c27_missing|_c28_missing|_c29_missing|_c30_missing|_c31_missing|_c32_missing|_c33_missing|_c34_missing|_c35_missing|_c36_missing|_c37_missing|_c38_missing|_c39_missing|
+-----------+-----------+-----------+-----------+-----------+-----------

In [116]:
df.count()

539

In [117]:
for i in range(20, 39):
    df = df.withColumn("_c"+str(i), df["_c"+str(i)].cast(FloatType()))

In [128]:
pipeline = Pipeline(stages=[
    StringIndexer(inputCol=c, outputCol='{}_index'.format(c), handleInvalid="keep")
    for c in ["_c"+str(i) for i in range(4, 20)] + ["_c39"]
])
model = pipeline.fit(df)
transformed = model.transform(df)

In [129]:
for c in ["_c"+str(i) for i in range(4, 20)] + ["_c39"]:
    transformed = transformed.drop(c)

In [130]:
transformed.printSchema()

root
 |-- _c20: float (nullable = true)
 |-- _c21: float (nullable = true)
 |-- _c22: float (nullable = true)
 |-- _c23: float (nullable = true)
 |-- _c24: float (nullable = true)
 |-- _c25: float (nullable = true)
 |-- _c26: float (nullable = true)
 |-- _c27: float (nullable = true)
 |-- _c28: float (nullable = true)
 |-- _c29: float (nullable = true)
 |-- _c30: float (nullable = true)
 |-- _c31: float (nullable = true)
 |-- _c32: float (nullable = true)
 |-- _c33: float (nullable = true)
 |-- _c34: float (nullable = true)
 |-- _c35: float (nullable = true)
 |-- _c36: float (nullable = true)
 |-- _c37: float (nullable = true)
 |-- _c38: float (nullable = true)
 |-- _c4_index: double (nullable = false)
 |-- _c5_index: double (nullable = false)
 |-- _c6_index: double (nullable = false)
 |-- _c7_index: double (nullable = false)
 |-- _c8_index: double (nullable = false)
 |-- _c9_index: double (nullable = false)
 |-- _c10_index: double (nullable = false)
 |-- _c11_index: double (nullable =

In [131]:
for i in list(range(4, 20))+[39]:
    transformed = transformed.withColumn("_c"+str(i)+"_index", transformed["_c"+str(i)+"_index"].cast(IntegerType()))

In [136]:
data = transformed.toPandas()
data.to_csv("./transformed.csv", header=False)